In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel

import numpy as np

# Creating a simple dataset with numerical and categorical features
np.random.seed(42)

# Numerical features
numerical_data = pd.DataFrame(np.random.rand(100, 3), columns=['num_feat1', 'num_feat2', 'num_feat3'])

# Categorical features
categories = ['category_A', 'category_B', 'category_C']
categorical_data = pd.DataFrame(np.random.choice(categories, size=(100, 2), p=[0.4, 0.4, 0.2]), columns=['cat_feat1', 'cat_feat2'])

# Target variable
target = np.random.choice([0, 1], size=(100,), p=[0.3, 0.7])

# Combine numerical, categorical, and target variables
df = pd.concat([numerical_data, categorical_data, pd.Series(target, name='target')], axis=1)

# Introduce some missing values
df.iloc[::5, 1] = np.nan
df.iloc[::10, 4] = np.nan

# Save the dataset to a CSV file
df.to_csv('your_dataset.csv', index=False)


# Load your dataset (replace 'your_dataset.csv' with your actual dataset file)
df = pd.read_csv('your_dataset.csv')

# Assume 'target' is your target variable
X = df.drop('target', axis=1)
y = df['target']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numerical features and categorical features
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine numerical and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

# Feature selection using RandomForestClassifier after encoding
feature_selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold='median')

# Final pipeline with feature selection and model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selector),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Evaluate the accuracy on the test dataset
accuracy = pipeline.score(X_test, y_test)
print(f'Accuracy on the test dataset: {accuracy:.2f}')


Accuracy on the test dataset: 0.80


In [4]:




# Import necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load the Iris dataset (replace with your dataset)
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create individual classifiers
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
lr_classifier = LogisticRegression(random_state=42)

# Create a pipeline for Random Forest Classifier
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', rf_classifier)
])

# Create a pipeline for Logistic Regression Classifier
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', lr_classifier)
])

# Create a Voting Classifier combining both classifiers
voting_classifier = VotingClassifier(
    estimators=[
        ('random_forest', rf_pipeline),
        ('logistic_regression', lr_pipeline)
    ],
    voting='hard'  # 'hard' for majority voting, 'soft' for weighted voting based on class probabilities
)

# Fit the Voting Classifier on the training data
voting_classifier.fit(X_train, y_train)

# Evaluate the accuracy on the test dataset
y_pred = voting_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy of the Voting Classifier: {accuracy:.2f}')


Accuracy of the Voting Classifier: 1.00
